# Imports

In [ ]:
import warnings

import awkward as ak
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import ruptures as rpt
import uproot
from IPython.display import clear_output
from itables import init_notebook_mode
from matplotlib.ticker import AutoMinorLocator, MaxNLocator
from scipy import stats
from skimage.measure import LineModelND, ransac
from sklearn.cluster import DBSCAN
from sklearn.linear_model import LinearRegression, RANSACRegressor
from skspatial.objects import Cylinder, Line
from tqdm.auto import tqdm, trange

init_notebook_mode(all_interactive=True)

# Parameters

In [ ]:
# Save plots as figures
show_figures = True

# Events to process
# event_list = np.arange(0, 101, 1)
event_list = np.array([19, 33])

# Units for plot labels
q_unit = "?"
xy_unit = "mm"
z_unit = "?"
dh_unit = "?" if z_unit != xy_unit else xy_unit
time_unit = "ns"

# DBSCAN parameters for clustering
min_samples = 2
xy_epsilon = 8  # 8 ideal
z_epsilon = 8  # 8 ideal

# Cylinder parameters for dQ/dx
min_dA = np.sqrt(xy_epsilon**2 + z_epsilon**2) / 4
dh = np.sqrt(xy_epsilon**2 + z_epsilon**2) / 2

# RANSAC parameters for line fitting
residual_threshold = 6  # 6 ideal
max_trials = 1000

# File loading

In [ ]:
with uproot.open("rwf_0cd913fa_20230706_191437.data_array.root") as f:
    tree = f["rwf_array"]
    light_df = pd.DataFrame()
    for idx, arrays in enumerate(tree.iterate(library="np")):
        # print(idx)
        df = pd.DataFrame.from_dict(arrays, orient="index").T
        light_df = pd.concat([light_df, df], ignore_index=True)

In [ ]:
# Clean up the dataframe
# light_df = light_df.drop_duplicates(subset=['event', 'ch'], keep = "first")

In [ ]:
with uproot.open("evd_self_trigger-packets-2023_07_06_19_14_CEST.root") as f:
    charge_df = f["events"].arrays(library="pd").set_index("eventID")

# File verification

In [ ]:
charge_df.columns

In [ ]:
light_df.columns

In [ ]:
light_df[(light_df["event"] == 19) & (light_df["ch"] == 1)]["rwf"]

# Offset correction

In [ ]:
# Charge correction
global_q_offset = charge_df["event_hits_q"].apply(tuple).explode().min()
charge_df["event_hits_q"] = charge_df["event_hits_q"] - global_q_offset
charge_df["event_q"] = charge_df["event_hits_q"].apply(sum)
print(global_q_offset)

In [ ]:
# Z correction
global_z_min = charge_df["event_hits_z"].apply(tuple).explode().min()
global_z_max = charge_df["event_hits_z"].apply(tuple).explode().max()
print(global_z_max, global_z_min)

# Histograms

In [ ]:
print("Trigger time distribution")
charge_df["trig_time"].apply(np.mean).hist()

In [ ]:
print(f"Event duration in {time_unit}")
charge_df["event_duration"].hist()

In [ ]:
print(f"Charge per hit in {q_unit}")
(charge_df["event_q"] / charge_df["event_nhits"]).hist(bins=50)

In [ ]:
print(f"Charge per hit per event in {q_unit}")
(charge_df["event_q"] / charge_df["event_nhits"]).to_frame().reset_index().plot.scatter(
    x="eventID", y=0
)

In [ ]:
print(f"Event charge in {q_unit}")
charge_df["event_q"].hist()

In [ ]:
print(f"Hits q in {q_unit}")
charge_df["event_hits_q"].apply(tuple).explode().hist()

In [ ]:
print(f"Hits z in {z_unit}")
charge_df["event_hits_z"].apply(tuple).explode().hist()

# Functions

In [ ]:
# Create a list of holes
def generate_dead_area(z_range):
    # Dead area on chips 44, 54, 64
    fake_x1, fake_y1, fake_z1 = np.meshgrid(
        np.linspace(36, 60, 6),
        np.concatenate(
            [
                np.linspace(-76, -52, 6),
                np.linspace(-44, -20, 6),
                np.linspace(-12, 12, 6),
            ]
        ),
        z_range,
    )

    # Dead area on chip 33
    fake_x2, fake_y2, fake_z2 = np.meshgrid(
        np.linspace(4, 28, 6), np.linspace(20, 44, 6), z_range
    )

    # Dead area on chip 42
    fake_x3, fake_y3, fake_z3 = np.meshgrid(
        np.linspace(-28, -4, 6), np.linspace(-12, 12, 6), z_range
    )
    mask = fake_x3 + (fake_y3 + 16) <= 0
    fake_x3, fake_y3, fake_z3 = fake_x3[mask], fake_y3[mask], fake_z3[mask]

    fake_x4, fake_y4, fake_z4 = np.meshgrid([-14], [2], z_range)

    # Dead area on SiPMs
    fake_x5 = []
    fake_y5 = []
    fake_z5 = []
    for k in range(4):
        for l in range(5):
            if (k == 3 and l < 3) or (k == 2 and l == 3) or (k == 1 and l == 2):
                continue

            temp_x, temp_y, temp_z = np.meshgrid(
                np.array([-50, -46]) + 32 * k,
                np.array([-66, -62]) + 32 * l,
                z_range,
            )

            fake_x5.extend(temp_x)
            fake_y5.extend(temp_y)
            fake_z5.extend(temp_z)

    fake_x5 = np.array(fake_x5)
    fake_y5 = np.array(fake_y5)
    fake_z5 = np.array(fake_z5)

    # Concatenate all the fake data
    fake_x = np.concatenate(
        [
            fake_x1.flatten(),
            fake_x2.flatten(),
            fake_x3.flatten(),
            fake_x4.flatten(),
            fake_x5.flatten(),
        ]
    )
    fake_y = np.concatenate(
        [
            fake_y1.flatten(),
            fake_y2.flatten(),
            fake_y3.flatten(),
            fake_y4.flatten(),
            fake_y5.flatten(),
        ]
    )
    fake_z = np.concatenate(
        [
            fake_z1.flatten(),
            fake_z2.flatten(),
            fake_z3.flatten(),
            fake_z4.flatten(),
            fake_z5.flatten(),
        ]
    )

    fake_data = np.c_[fake_x, fake_y, fake_z]

    return fake_data


# Apply DBSCAN clustering
def cluster(hitArray):
    # First stage clustering
    z_intervals = []
    first_stage = DBSCAN(eps=xy_epsilon, min_samples=min_samples).fit(hitArray[:, 0:2])
    for label in first_stage.labels_:
        if label > -1:
            mask = first_stage.labels_ == label
            z = hitArray[mask, 2]
            z_intervals.append((min(z), max(z)))

    # Sort the intervals based on their start points
    sorted_intervals = sorted(z_intervals, key=lambda interval: interval[0])

    # Initialize a list to store the intervals representing the empty space
    empty_space_ranges = []

    # Iterate through the sorted intervals to find the gaps
    for i in range(len(sorted_intervals) - 1):
        current_interval = sorted_intervals[i]
        next_interval = sorted_intervals[i + 1]

        # Calculate the gap between the current interval and the next interval
        gap_start = current_interval[1]
        gap_end = next_interval[0]

        # Check if there is a gap (empty space) between intervals
        if gap_end > gap_start and gap_end < gap_start + 40:
            empty_space_ranges.append(np.arange(gap_start, gap_end, z_epsilon))

    if not empty_space_ranges:
        empty_space_ranges.append(
            np.arange(
                np.mean(hitArray[:, 2]) - np.std(hitArray[:, 2]),
                np.mean(hitArray[:, 2]) + np.std(hitArray[:, 2]),
                z_epsilon,
            )
        )

    z_range = np.concatenate(empty_space_ranges)

    # Create a list of holes
    fake_data = generate_dead_area(z_range)
    fake_data_count = len(fake_data)

    # Second stage clustering
    # Combine fake to true data
    second_stage_data = np.concatenate([hitArray, fake_data])
    second_stage = DBSCAN(eps=xy_epsilon, min_samples=1).fit(second_stage_data[:, 0:2])

    # Third stage clustering
    # Create a new array with z and labels
    third_stage_z = np.c_[second_stage.labels_ * 1e3, second_stage_data[:, 2]]
    flag = second_stage.labels_ > -1
    third_stage_data = third_stage_z[flag].copy()
    third_stage = DBSCAN(
        eps=z_epsilon, min_samples=min_samples, metric="chebyshev"
    ).fit(third_stage_data)

    # Remove fake data
    # Shift labels by 1 so that negative values are reserved for outliers
    labels = third_stage.labels_[:-fake_data_count] + 1

    return labels


# Apply Ransac Fit
def ransacFit(hitArray, weightArray=None):
    if weightArray is not None:
        estimator = RANSACRegressor(
            min_samples=min_samples,
            max_trials=max_trials,
            residual_threshold=residual_threshold,
        )
        inliers = estimator.fit(
            hitArray[:, 0:2],
            hitArray[:, 2],
            sample_weight=weightArray,
        ).inlier_mask_

        score = estimator.score(hitArray[:, 0:2], hitArray[:, 2])
    else:
        model_robust, inliers = ransac(
            hitArray,
            LineModelND,
            min_samples=min_samples,
            residual_threshold=residual_threshold,
            max_trials=max_trials,
        )

        score = model_robust.score(hitArray)

    outliers = inliers == False
    return inliers, outliers, score


# Apply best line fit
def lineFit(hitArray):
    line = Line.best_fit(hitArray)
    residuals = []
    for point in hitArray:
        distance = line.distance_point(point)
        residuals.append(distance)

    # Convert residuals to a numpy array
    residuals = np.array(residuals)

    # Calculate chi-squared
    chi_squared = np.sum(residuals**2)

    return line, chi_squared


# Calculate dQ/dx from a line fit
def dqdx(hitArray, q, line_fit, dh, dA, h, ax=None):
    # Cylinder step size for dQ/dx
    steps = np.arange(-2 * dh, h + 3 * dh, dh)

    # Mask of points that have been accounted for
    counted = np.zeros(len(q), dtype=bool)

    # Array of dQ values for each step
    dq_i = np.zeros(len(steps), dtype=float)

    for step_idx, step in enumerate(steps):
        cylinder_fit = Cylinder(
            line_fit.to_point(h / 2 - step),
            -line_fit.direction.unit() * dh,
            dA,
        )
        if ax is not None:
            cylinder_fit.plot_3d(ax)

        for point_idx, point in enumerate(hitArray):
            if not counted[point_idx] and cylinder_fit.is_point_within(point):
                counted[point_idx] = True
                dq_i[step_idx] += q[point_idx]

    return dq_i


# Fit clusters with Ransac method
def fitClusters(hitArray, q, labels, ax2d, ax3d, plot_cyl=False, refit_outliers=True):
    # Create a dictionary of lines
    lines = {}
    # Create a dictionary of p-values
    p_values = {}
    # Create a dictionary of scores
    scores = {}
    # Create a dictionary of dq
    dq = {}

    # Fit clusters
    idx = 0
    condition = lambda: idx < len(np.unique(labels))
    while condition():
        label = np.unique(labels)[idx]
        mask = labels == label
        if label > 0 and mask.sum() > min_samples:
            XYZ_c = hitArray[mask]
            x_c, y_c, z_c = XYZ_c[:, 0], XYZ_c[:, 1], XYZ_c[:, 2]
            q_c = np.array(q)[mask]

            norm = np.linalg.norm(
                [
                    x_c.max() - x_c.min(),
                    y_c.max() - y_c.min(),
                    z_c.max() - z_c.min(),
                ]
            )

            # Fit the model
            inliers, outliers, score = ransacFit(XYZ_c, q_c - min(q_c) + 1)
            scores[label] = score

            # Refit outliers
            level_1 = np.where(mask)[0]
            level_2 = np.where(outliers)[0]
            level_3 = level_1[level_2]

            if refit_outliers and sum(outliers) > min_samples:
                outlier_labels = cluster(XYZ_c[outliers])
                last_label = max(labels) + 1
                # Assign positive labels to clustered outliers and negative labels to unlclustered outliers
                for i, j in enumerate(level_3):
                    labels[j] = (outlier_labels[i] + last_label) * (
                        1 if outlier_labels[i] > 0 else -1
                    )
            else:
                # Assign negative labels to outliers
                for j in level_3:
                    labels[j] = -labels[j]

            if sum(inliers) > min_samples:
                line_fit, chi_squared = lineFit(XYZ_c[inliers])
                # Degrees of freedom (number of points - number of parameters in the line fit)
                degrees_of_freedom = sum(inliers) - 1
                # Calculate the reduced chi squared
                reduced_chi_squared = chi_squared / degrees_of_freedom
                # Calculate the p-value (assuming chi-squared distribution)
                p_value = 1 - stats.chi2.cdf(chi_squared, degrees_of_freedom)

                lines[label] = line_fit
                p_values[label] = p_value

                if ax2d is not None:
                    # 2D plot
                    line_fit.plot_2d(
                        ax2d,
                        t_1=-norm / 2,
                        t_2=norm / 2,
                        c="red",
                        label="Fit",
                        zorder=10,
                    )
                if ax3d is not None:
                    # 3D plot
                    line_fit.plot_3d(
                        ax3d, t_1=-norm / 2, t_2=norm / 2, c="red", label="Fit"
                    )

                # Calculate dQ/dx
                dA = max(np.sqrt(reduced_chi_squared), min_dA)
                dq_i = dqdx(
                    XYZ_c[inliers],
                    q_c[inliers],
                    line_fit,
                    dh=dh,
                    dA=dA,
                    h=norm,
                    ax=ax3d if ax3d is not None and plot_cyl else None,
                )

                if dq_i.sum() != 0:
                    dq[label] = dq_i
                else:
                    dq[label] = 0

        idx += 1

    metrics = {
        "DBSCAN_labels": labels,
        "Fit_lines": lines,
        "Fit_p_values": p_values,
        "RANSAC_scores": scores,
        "dQ": dq,
    }

    return metrics

In [ ]:
def create_axes(event_idx, charge):
    fig = plt.figure(figsize=(12, 6))
    ax3d = fig.add_subplot(121, projection="3d")
    ax2d = fig.add_subplot(122)
    fig.suptitle(f"Event {event_idx} - Charge = {charge} ?")

    # Draw dead areas
    for i in range(4):
        j = 0
        if i == 3:
            j = 1
        ax2d.plot(
            np.arange(32, 64 + 1, 1) - 32 * j,
            np.arange(-80, -48 + 1, 1) + 32 * i,
            c="white",
            lw=1,
        )
        ax2d.plot(
            np.arange(64, 32 - 1, -1) - 32 * j,
            np.arange(-80, -48 + 1, 1) + 32 * i,
            c="white",
            lw=1,
        )

    # ax2d.plot(np.linspace(-31,0),np.linspace(8,-16), c="white",lw = 1)
    ax2d.plot(np.linspace(-32, 0), np.linspace(16, -16), c="white", lw=1, zorder=-1)
    ax2d.plot(np.linspace(-32, -16), np.linspace(-16, 0), c="white", lw=1, zorder=-1)

    # Adjust axes
    for ax in [ax3d, ax2d]:
        ax.set_aspect("equal", adjustable="box")
        ax.set_xlim([-64, 64])
        ax.set_ylim([-80, 80])
        ax.set_xlabel(f"x [{xy_unit}]")
        ax.set_ylabel(f"y [{xy_unit}]")
        ax.set_xticks(np.linspace(-64, 64, 5))
        ax.set_yticks(np.linspace(-80, 80, 6))
        ax.xaxis.set_minor_locator(AutoMinorLocator(8))
        ax.yaxis.set_minor_locator(AutoMinorLocator(8))
        ax.grid()

    # Adjust z-axis
    ax3d.set_zlabel(f"z [{z_unit}]")
    ax3d.zaxis.set_minor_locator(AutoMinorLocator(8))
    ax3d.zaxis.set_major_locator(MaxNLocator(integer=True))

    return fig, (ax2d, ax3d)


def event_display(event_idx, x, y, z, t, q, plot_cyl=False, plot_line=True):
    if len(x) < 2:
        return None, None, None, None, None

    # Plot the hits
    fig, axes = create_axes(event_idx, round(sum(q)))
    ax2d = axes[0]
    ax3d = axes[1]

    # Group by x and y coordinates and sum the z values
    data2d = np.c_[x, y, q]
    unique_points, indices = np.unique(data2d[:, :2], axis=0, return_inverse=True)
    q_sum = np.bincount(indices, weights=data2d[:, 2])

    # Plot the hits
    plot3d = ax3d.scatter(
        x, y, z, c=q, marker="s", s=20, vmin=q_sum.min(), vmax=q_sum.max()
    )
    plot2d = ax2d.scatter(
        unique_points[:, 0],
        unique_points[:, 1],
        c=q_sum,
        marker="s",
        s=30,
        vmin=q_sum.min(),
        vmax=q_sum.max(),
    )
    cbar = plt.colorbar(plot2d)
    cbar.set_label(f"charge [{q_unit}]")

    # Create a design matrix
    XYZ = np.c_[x, y, z]

    # Cluster the hits
    labels = cluster(XYZ)

    # Fit clusters
    metrics = fitClusters(XYZ, q, labels, ax2d, ax3d, plot_cyl)

    # Draw the SIPMs
    for k in range(4):
        for l in range(5):
            vertices_x = np.array([-51, -51, -45, -45, -51])
            vertices_y = np.array([-67, -61, -61, -67, -67])
            ax2d.fill(k * 32 + vertices_x, l * 32 + vertices_y, c="white", zorder=5)

    # ax3d.view_init(160, 110, -85)
    ax3d.view_init(30, 20, 100)
    # ax3d.view_init(0, 0, 0)
    # ax3d.view_init(0, 0, 90)
    fig.tight_layout()

    fig.savefig(f"event_{event_idx}.png", dpi=300)
    # plt.show()

    return metrics


# Plot dQ versus X
def plot_dQ(dQ_array, event_idx, track_idx, dx):
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax_twinx = ax.twinx()

    fig.suptitle(
        f"Event {event_idx} - Track {track_idx} - dx = {round(dh,2)} [{dh_unit}]"
    )

    mean = np.mean(dQ_array[dQ_array > 0])
    ax.axhline(mean, ls="--", c="red", label=f"Mean = {round(mean,2)} [{q_unit}]", lw=1)

    ax.step(np.arange(0, (len(dQ_array)) * dx, dx), dQ_array)
    ax.set_xlabel(f"x [{dh_unit}]")
    ax.set_ylabel(f"dQ [{q_unit}]")

    ax_twinx.step(
        np.arange(0, (len(dQ_array)) * dx, dx), np.cumsum(dQ_array), color="C1"
    )
    ax_twinx.set_ylabel(f"Q [{q_unit}]")

    for axes in [ax, ax_twinx]:
        axes.xaxis.set_minor_locator(AutoMinorLocator())
        axes.yaxis.set_minor_locator(AutoMinorLocator())
        axes.xaxis.set_major_locator(MaxNLocator())
        axes.yaxis.set_major_locator(MaxNLocator())

    ax.legend(loc="upper left")

    fig.tight_layout()
    fig.savefig(f"dQ_{event_idx}_{track_idx}_{round(dh,2)}.png")

# Event display

## Fake data map

In [ ]:
def plot_fake_data(z_range):
    fake_data = generate_dead_area(z_range)
    fake_x, fake_y, fake_z = fake_data[:, 0], fake_data[:, 1], fake_data[:, 2]

    fig = plt.figure()

    ax = fig.add_subplot(111)
    ax.set_aspect("equal", adjustable="box")
    ax.scatter(fake_x, fake_y, marker="s", s=20)
    ax.set_xlim([-64, 64])
    ax.set_ylim([-80, 80])
    ax.set_xlabel(f"x [{xy_unit}]")
    ax.set_ylabel(f"y [{xy_unit}]")
    ax.set_xticks(np.linspace(-64, 64, 5))
    ax.set_yticks(np.linspace(-80, 80, 6))
    ax.xaxis.set_minor_locator(AutoMinorLocator(8))
    ax.yaxis.set_minor_locator(AutoMinorLocator(8))
    ax.grid()

    fig.savefig("fake_data_map.png", dpi=300)


plot_fake_data(np.arange(-300, 0, 10))

plt.show()

## Real data + fit

In [ ]:
# Suppress the UndefinedMetricWarning
warnings.filterwarnings("ignore", category=Warning)

In [ ]:
metrics = {}

for idx in tqdm(event_list):
    row = charge_df.loc[idx]

    x = np.array(row["event_hits_x"])
    y = np.array(row["event_hits_y"])
    z = np.array(row["event_hits_z"])
    t = np.array(row["event_hits_ts"])
    q = np.array(row["event_hits_q"])

    non_zero_indices = (x != 0) * (y != 0)

    # Apply boolean indexing to x, y, and z arrays
    x = x[non_zero_indices]
    y = y[non_zero_indices]
    z = z[non_zero_indices]
    t = t[non_zero_indices]
    q = q[non_zero_indices]

    if len(x) > 2:
        metrics[idx] = event_display(
            idx,
            x,
            y,
            z,
            t,
            q,
            plot_cyl=True,
        )
        if show_figures:
            plt.show()
        else:
            plt.close()

    else:
        print("Not enough hits to fit")

In [ ]:
# Reset the warning filter (optional)
warnings.filterwarnings("default", category=Warning)

## Metrics

In [ ]:
metrics

# dQ/dx

In [ ]:
event_idx = 19
track_idx = 1

In [ ]:
print("Track total charge =", np.sum(metrics[event_idx]["dQ"][track_idx]))

In [ ]:
print("Track dQ per dx")
print(metrics[event_idx]["dQ"][track_idx])

In [ ]:
for event_idx in tqdm(event_list, leave=False):
    for track_idx, dQ_array in metrics[event_idx]["dQ"].items():
        if track_idx > 0:
            plot_dQ(dQ_array, event_idx, track_idx, dh)

            if show_figures:
                plt.show()
            else:
                plt.close()

# Light wave forms

In [ ]:
event_idx = 19

for ch_idx in tqdm(sorted(light_df["ch"].unique())):
    rows = light_df[(light_df["event"] == event_idx) & (light_df["ch"] == ch_idx)][
        "rwf"
    ]
    for row in rows:
        if len(row) == 0:
            continue

        fig = plt.figure()
        ax = plt.subplot(111)

        fig.suptitle(f"Event {event_idx} - Channel {ch_idx}")
        ax.plot(row[:-1])

        plt.show()